<a href="https://colab.research.google.com/github/jasminiballa4/app/blob/main/agents/langgraph_basic_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_core langchain_openai langchain_community langgraph google-search-results

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.0 MB/s eta 0:00:00
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=d31c8c80f87a458e4652d6f4b263a8bf6601bbc33845c3104cef0da5203010d2
  Stored in directory: /root/.cache/pip/wheels/0c/47/f5/89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the pa

In [7]:
import os
import operator
from typing import TypedDict, Annotated, List
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SerpAPIWrapper
from langgraph.graph import StateGraph, END

In [8]:
#os.environ["SERPAPI_API_KEY"] = "api-key"

llm = ChatOpenAI(openai_api_base = "https://openrouter.ai/api/v1", openai_api_key = "API-KEY", model = "arcee-ai/trinity-large-preview:free",temperature=0.9) #max_tokens=100

In [9]:
# --- Tool Definition ---
# Instantiate the SerpAPI search tool.
search_tool = SerpAPIWrapper()

# --- State Definition ---
# This defines the "memory" or "state" that flows through the graph.
class ResearchState(TypedDict):
    topic: str
    explanation: str
    summary: str


In [10]:
def researcher_agent(state: ResearchState) -> dict:
    """
    This agent uses a web search tool to find information on a topic and then explains it.
    """
    print("---RESEARCHER (with SerpApi Web Search)---")
    topic = state["topic"]

    # Use the tool to search for information. The .run() method takes the query string.
    search_results = search_tool.run(topic)

    print(f"Search Results:\n{search_results}")

    prompt = ChatPromptTemplate.from_template(
        """You are a helpful research assistant. Based on the following search results,
        provide a brief, easy-to-understand explanation of the topic: {topic}.

        Search Results:
        {search_results}
        """
    )

    chain = prompt | llm
    result = chain.invoke({"topic": topic, "search_results": search_results})

    print(f"Researcher's Explanation:\n{result.content}")
    return {"explanation": result.content}

In [11]:
def summarizer_agent(state: ResearchState) -> dict:
    """
    This agent takes an explanation and summarizes it in one sentence.
    """
    print("---SUMMARIZER---")
    explanation = state["explanation"]

    prompt = ChatPromptTemplate.from_template(
        "You are a summarization expert. Condense the following text into a single, concise sentence:\n\n{explanation}"
    )

    chain = prompt | llm
    result = chain.invoke({"explanation": explanation})

    print(f"Summarizer's Output:\n{result.content}")
    return {"summary": result.content}

In [12]:
# --- Graph Definition ---

workflow = StateGraph(ResearchState)

# Add the agent functions as nodes
workflow.add_node("researcher", researcher_agent)
workflow.add_node("summarizer", summarizer_agent)

# Define the edges, which control the flow
workflow.add_edge("researcher", "summarizer")
workflow.add_edge("summarizer", END)

# Set the entry point
workflow.set_entry_point("researcher")

# Compile the graph
app = workflow.compile()

In [ ]:
# --- Main Execution Block ---
if __name__ == "__main__":
    topic = input("Please enter a topic for the agents to research and summarize: ")

    inputs = {"topic": topic}
    final_state = app.invoke(inputs)

    print("\n--- FINAL SUMMARY ---")
    print(final_state['summary'])